In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
from timeit import default_timer as timer
from _collections import OrderedDict
import shutil
from pprint import pprint
start = timer()
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.file_location import FileLocationManager
from utilities.sqlcontroller import SqlController
from utilities.utilities_cvat_neuroglancer import get_structure_number, NumpyToNeuroglancer, get_segment_properties
from utilities.utilities_affine import align_point_sets, load_atlas_data, estimate_structure_centers


Connecting dklab@192.168.1.12:3306


In [2]:
animal = 'DK52'
fileLocationManager = FileLocationManager(animal)
OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'atlas')
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

atlas_data = load_atlas_data()
source_centers = estimate_structure_centers(atlas_data, animal)
# DK52 structure centers
output_centers = {
    '12N': [46488, 18778, 242],
    '5N_L': [38990, 20019, 172],
    '5N_R': [39184, 19027, 315],
    '7N_L': [42425, 23190, 166],
    '7N_R': [42286, 22901, 291]
}
structures = sorted(output_centers.keys())
source_point_set = np.array([source_centers[s][0] for s in structures]).T
# Unify to 1 mum scale in all axes
source_scale = np.diagflat([10, 10, 20])
source_point_set = source_scale @ source_point_set
output_point_set = np.array([output_centers[s] for s in structures]).T

# Unify to 1 mum scale in all axes
output_scale = np.diagflat([0.325, 0.325, 20])
output_point_set = output_scale @ output_point_set

r_auto, t_auto = align_point_sets(source_point_set, output_point_set)
t_auto_out = (np.linalg.inv(output_scale) @ t_auto).T


resolution: 0.325
width: 65000
height: 36000
box center: [1056.25  585.    242.5 ]


In [3]:
sql_controller = SqlController(animal)
thumbnail_dir = os.path.join(fileLocationManager.prep, 'CH1', 'thumbnail_aligned')

print(f'resolution: {sql_controller.scan_run.resolution}')
print(f'width: {sql_controller.scan_run.width}')
print(f'height: {sql_controller.scan_run.height}')
box_w = sql_controller.scan_run.width * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_h = sql_controller.scan_run.height * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_z = len(os.listdir(thumbnail_dir))  # 20 mum scale
box_center = np.array([box_w, box_h, box_z]) / 2

atlasV7_volume = np.zeros((int(box_h), int(box_w), int(box_z)), dtype=np.uint8)
print('Shape of atlas volume', atlasV7_volume.shape)


resolution: 0.325
width: 65000
height: 36000
Shape of atlas volume (1170, 2112, 485)


In [4]:
print(t_auto_out)
print(output_scale)

[[17580.00383156 15298.3395672     88.21494223]]
[[ 0.325  0.     0.   ]
 [ 0.     0.325  0.   ]
 [ 0.     0.    20.   ]]


In [18]:
for structure, (source_point, volume) in sorted(source_centers.items()):
    print(str(structure).ljust(6),end=": ")
    results = r_auto @ source_point + t_auto.T
    x = results[0][0]
    y = results[0][1]
    z = results[0][1]
    print(x,y,z)


10N_L : 6615.742227761945 5125.862802706045 5125.862802706045
10N_R : 6616.073146143126 5124.006788058699 5124.006788058699
12N   : 6609.260820804933 5128.253853417324 5128.253853417324
3N_L  : 6359.632051831818 5081.840090697307 5081.840090697307
3N_R  : 6359.784688614908 5080.984000128606 5080.984000128606
4N_L  : 6365.719746152325 5077.491787952871 5077.491787952871
4N_R  : 6365.998936219433 5075.925900763121 5075.925900763121
5N_L  : 6437.24174284032 5145.362171820017 5145.362171820017
5N_R  : 6438.8001938149955 5136.621322110057 5136.621322110057
6N_L  : 6456.025082344031 5123.081973038645 5123.081973038645
6N_R  : 6456.439965443732 5120.755027320995 5120.755027320995
7N_L  : 6510.933338534975 5193.430775843256 5193.430775843256
7N_R  : 6512.252201181017 5186.0336999233505 5186.0336999233505
7n_L  : 6479.739315556657 5166.395660705302 5166.395660705302
7n_R  : 6480.915421845872 5159.799259224598 5159.799259224598
AP    : 6581.298455091598 5098.565972472305 5098.565972472305
Amb_L 

In [44]:
z_length = 486
volume_shape = 337
SC_start = 127
SC_end = 311
midz = 238
z_origin = -185
midz - hv

153.75

In [52]:
print('end - start=',SC_end - SC_start)
print('half of volume=',volume_shape/2)
print('find this=', midz - SC_start )
z_start = (midz/2 - volume_shape/2)
z_end = z_start + (volume_shape + 1) // 2

print('start, end', z_start, z_end)
print('volume shape and target length', volume_shape, z_end-z_start)

end - start= 184
half of volume= 168.5
find this= 111
start, end -49.5 119.5
volume shape and target length 337 169.0


In [7]:
%%time
#ng = NumpyToNeuroglancer(atlasV7_volume, [10000, 10000, 20000])
#ng.preview()
#ng.init_precomputed(OUTPUT_DIR)
#ng.add_segment_properties(get_segment_properties())
#ng.add_downsampled_volumes()
#ng.add_segmentation_mesh()

CPU times: user 7 µs, sys: 2 µs, total: 9 µs
Wall time: 16.2 µs
